## TENSORBOARD USED
- tensorboard --logdir =./logs

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

In [3]:
X = tf.placeholder(tf.float32, [None, 2], name='x-input')
Y = tf.placeholder(tf.float32, [None, 1], name='y-input')

In [4]:
with tf.name_scope("layer1") as scope:
    W1 = tf.Variable(tf.random_normal([2,2]),name="Weight_one")
    b1 = tf.Variable(tf.random_normal([2]),name="Bias_one")
    layer_one = tf.sigmoid(tf.matmul(X,W1) + b1)
    
    w1_histogram = tf.summary.histogram("Weight_one", W1)
    b1_histogram = tf.summary.histogram("Bias-one",b1)
    layer_one_histogram = tf.summary.histogram("layer1",layer_one)

## more Deep 

In [5]:
with tf.name_scope("layer2") as scope:
    W2 = tf.Variable(tf.random_normal([2,1]),name="Weight_two")
    b2 = tf.Variable(tf.random_normal([1]),name="Bias_two")
    Hypothesis = tf.sigmoid(tf.matmul(layer_one,W2) + b2)
    
    w2_histogram = tf.summary.histogram("Weight_two", W2)
    b2_histogram = tf.summary.histogram("Bias-two",b2)
    layer_two_histogram = tf.summary.histogram("Hypothesis",Hypothesis)

In [6]:
with tf.name_scope("cost") as scope:
    cost = -tf.reduce_mean(Y * tf.log(Hypothesis) + (1 - Y) * tf.log(1 - Hypothesis))
    cost_summ = tf.summary.scalar("cost", cost)

In [7]:
with tf.name_scope("train") as scope:
    train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

In [8]:
predicted = tf.cast(Hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
accuracy_summ = tf.summary.scalar("accuracy", accuracy)

In [9]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [10]:
merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("\\xor_logs_01")
writer.add_graph(sess.graph)

In [11]:
for step in range(10001):
        summary, _ = sess.run([merged_summary, train], feed_dict={X: x_data, Y: y_data})
        writer.add_summary(summary, global_step=step)

        if step % 500 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}), sess.run([W1, W2]))

0 1.90726 [array([[-1.06297135, -0.63679367],
       [ 0.1662657 ,  0.78239495]], dtype=float32), array([[ 1.16057563],
       [ 1.42459059]], dtype=float32)]
500 0.794502 [array([[-1.1623069 , -0.7964884 ],
       [ 0.07643434,  0.60981208]], dtype=float32), array([[ 0.76701856],
       [ 0.22666746]], dtype=float32)]
1000 0.695468 [array([[-1.18006957, -0.80023468],
       [ 0.09314632,  0.6055966 ]], dtype=float32), array([[ 0.66825688],
       [-0.10791894]], dtype=float32)]
1500 0.691774 [array([[-1.18456089, -0.79962265],
       [ 0.13183883,  0.61220658]], dtype=float32), array([[ 0.65492582],
       [-0.17958321]], dtype=float32)]
2000 0.691222 [array([[-1.1880312 , -0.80057836],
       [ 0.17466177,  0.61915445]], dtype=float32), array([[ 0.65917915],
       [-0.20383289]], dtype=float32)]
2500 0.690731 [array([[-1.19247603, -0.80205178],
       [ 0.21889681,  0.62661093]], dtype=float32), array([[ 0.66887712],
       [-0.2204092 ]], dtype=float32)]
3000 0.690202 [array([[-1.1

In [12]:
h, c, a = sess.run([Hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data})
print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)


Hypothesis:  [[ 0.49660048]
 [ 0.54613131]
 [ 0.47485587]
 [ 0.47427541]] 
Correct:  [[ 0.]
 [ 1.]
 [ 0.]
 [ 0.]] 
Accuracy:  0.75
